In [1]:
import sqlite3
import pandas as pd
import numpy as np

# Read data

In [2]:
# download the raw data from FAO websites
df = pd.read_csv('FoodBalanceSheets_E_All_Data/FoodBalanceSheets_E_All_Data_NOFLAG.csv', # https://www.fao.org/faostat/en/#data/FBS Bulk Downloads - All Data
                 encoding="ISO-8859-1")
dfh = pd.read_csv('FoodBalanceSheetsHistoric_E_All_Data/FoodBalanceSheetsHistoric_E_All_Data_NOFLAG.csv', # https://www.fao.org/faostat/en/#data/FBSH Bulk Downloads - All Data
                 encoding="ISO-8859-1")
dfh

,Area Code,Area Code (M49),Area,Item Code,Item Code (FBS),Item,Element Code,Element,Unit,Y1961,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,2,'004,Afghanistan,2501,'S2501,Population,511,Total Population - Both sexes,1000 No,8954.00,...,24019.00,24861.00,25631.00,26349.00,27032.00,27708.00,28398.00,29105.00,29825.00,30552.00
1,2,'004,Afghanistan,2901,'S2901,Grand Total,664,Food supply (kcal/capita/day),kcal/cap/d,2999.00,...,1967.00,1948.00,1966.00,2046.00,2041.00,2081.00,2104.00,2107.00,2100.00,2090.00
2,2,'004,Afghanistan,2901,'S2901,Grand Total,674,Protein supply quantity (g/capita/day),g/cap/d,84.91,...,55.24,53.51,53.46,56.00,56.96,57.79,58.14,58.91,58.91,58.25
3,2,'004,Afghanistan,2901,'S2901,Grand Total,684,Fat supply quantity (g/capita/day),g/cap/d,37.51,...,34.95,36.75,31.13,32.09,29.72,30.72,33.88,33.08,33.37,33.52
4,2,'004,Afghanistan,2903,'S2903,Vegetal Products,664,Food supply (kcal/capita/day),kcal/cap/d,2752.00,...,1726.00,1715.00,1762.00,1839.00,1831.00,1871.00,1888.00,1891.00,1883.00,1873.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238413,5817,'902,Net Food Importing Developing Countries,2680,'S2680,Infant food,674,Protein supply quantity (g/capita/day),g/cap/d,0.01,...,0.02,0.02,0.02,0.03,0.04,0.03,0.03,0.04,0.04,0.04
238414,5817,'902,Net Food Importing Developing Countries,2680,'S2680,Infant food,684,Fat supply quantity (g/capita/day),g/cap/d,0.00,...,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01
238415,5817,'902,Net Food Importing Developing Countries,2899,'S2899,Miscellaneous,664,Food supply (kcal/capita/day),kcal/cap/d,0.12,...,0.84,0.98,1.07,1.06,1.11,1.18,1.12,1.24,1.36,1.41
238416,5817,'902,Net Food Importing Developing Countries,2899,'S2899,Miscellaneous,674,Protein supply quantity (g/capita/day),g/cap/d,0.00,...,0.06,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07


## Pre-check

In [30]:
for code in dfh['Item Code']:
    if code not in df['Item Code']:
        print(code)

In [31]:
for code in dfh['Element Code']:
    if code not in df['Element Code']:
        print(code)

# FAO

In [3]:
fao = pd.read_excel('DataDict.xlsx')
print(fao.shape)
# fao.columns

(168, 8)


## Country

In [4]:
country = pd.read_excel('Country.xlsx')
countryH = pd.read_excel('CountryH.xlsx')

## OtherUtil

### Crops

In [6]:
fao_utl = fao[fao['DisplayNotes'] == 'Other']
dd = fao_utl[fao_utl['IFs Group'] == 'Crops']

cf_agg = pd.read_excel('FAO FBS aggregation.xlsx')
agg = cf_agg[cf_agg['IFs Group'] == 'Crops']

dt = df[df['Element Code'].isin([5154,5171])]
dth = dfh[dfh['Element Code'].isin([5154,5171])]

dt = dt[dt['Item Code'].isin(agg['Item Code'])]
dth = dth[dth['Item Code'].isin(agg['Item Code'])]

for j in dd.index:
    v = dd.loc[j,'Variable']
    # m = dd.loc[j,'Element Code']
    i = dd.loc[j,'Item Code']
    u = dd.loc[j,'Units']
    if dd.isnull().loc[j,'Formula']:
        f = dd.loc[j,'Formula']
    else:
        f = "val" + str(dd.loc[j,'Formula'])
    if dd.isnull().loc[j,'Decimal Places']:
        d = dd.loc[j,'Decimal Places']
    else:
        d = int(dd.loc[j,'Decimal Places'])
    print(v,u,f,d)
    # dt1 = dt[dt['Item Code'] == i]
    dt1 = dt[['Area','Y2010', 'Y2011', 'Y2012',
        'Y2013', 'Y2014', 'Y2015', 'Y2016', 'Y2017', 'Y2018', 'Y2019', 'Y2020',
        'Y2021']
        ]
    dt1.columns = ['Area', 2010, 2011, 2012,
    2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
    2021]
    dt1 = dt1.melt(
        id_vars='Area',
        var_name='year',
        value_name = 'val')
    
    if dd.isnull().loc[j,'Formula']:
        dt1 = dt1
    else:
        dt1.val = dt1.eval(f)
    if dd.isnull().loc[j,'Decimal Places']:
        dt1 = dt1
    else:
        dt1.val = round(dt1.val, d)
    # print(dt1.shape)
    dt1 = country.merge(dt1,
                        on = 'Area',
                        how = 'left')
    dt1.drop(columns='Area',
            inplace=True)
    # dth1 = dth[dth['Item Code'] == i]
    dth1 = dth[['Area','Y1961', 'Y1962', 'Y1963',
    'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971',
    'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979',
    'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
    'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995',
    'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003',
    'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009']
        ]
    dth1.columns = ['Area', 1961, 1962, 1963,
    1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
    1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
    1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987,
    1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
    1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
    2004, 2005, 2006, 2007, 2008, 2009]
    dth1 = dth1.melt(
        id_vars='Area',
        var_name='year',
        value_name = 'val')
    if dd.isnull().loc[j,'Formula']:
        dth1 = dth1
    else:
        dth1.val = dth1.eval(f)
    if dd.isnull().loc[j,'Decimal Places']:
        dth1 = dth1
    else:
        dth1.val = round(dth1.val, d)
    dth1 = countryH.merge(dth1,
                        on = 'Area',
                        how = 'left')
    dth1.drop(columns='Area',
            inplace=True)
    # print(dth1.shape)
    
    final = pd.concat([dt1,dth1])
    final = final.groupby(['Country', 'FIPS_CODE','year']).sum(min_count=1).reset_index()
    first_last = final.sort_values(['Country','year']).groupby(final['Country'])['val'] \
        .agg(['first', 'last']).reset_index()
    first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
    final = final.pivot_table(index = ['Country', 'FIPS_CODE'],
                    columns='year',
                    values = 'val').reset_index()
    final = final.merge(first_last,
                        on = 'Country',
                        how = 'left'
                        )
    final = country.merge(final,
                        on = ['Country','FIPS_CODE'],
                        how = 'left')
    final.drop(columns='Area', inplace = True)
    final.dropna(axis = 0, how = 'all', inplace=True)
    final = final.drop_duplicates(ignore_index=True)
    print(v)
    final = final.sort_values(by='Country')
    final = final.set_index(['Country','FIPS_CODE'])
    try:
        final = final.apply(lambda x: pd.Series.round(x, d))
    except:
        final = final
    # final.to_csv(f'IFs Import/{v}.csv',
    #             #  index=False
    #              )
final



AGCroptoOtherUtilFAO Tonnes val*1000 nan
AGCroptoOtherUtilFAO


,,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2014,2015,2016,2017,2018,2019,2020,2021,Earliest,MostRecent
Country,FIPS_CODE,,,,,,,,,,,,,,,,,,,,,
Afghanistan,AFG,5000.0,5000.0,5000.0,5000.0,6000.0,8000.0,14000.0,17000.0,16000.0,15000.0,...,27000.0,12000.0,29000.0,40000.0,40000.0,37000.0,34000.0,22000.0,5000.0,22000.0
Albania,ALB,6000.0,16000.0,2000.0,6000.0,6000.0,3000.0,2000.0,9000.0,2000.0,2000.0,...,158000.0,154000.0,146000.0,150000.0,124000.0,142000.0,148000.0,125000.0,6000.0,125000.0
Algeria,DZA,29000.0,3000.0,24000.0,17000.0,18000.0,18000.0,18000.0,24000.0,29000.0,22000.0,...,2126000.0,2767000.0,3070000.0,2740000.0,3410000.0,3395000.0,1674000.0,1846000.0,29000.0,1846000.0
Angola,AGO,61000.0,31000.0,36000.0,68000.0,50000.0,75000.0,9000.0,16000.0,19000.0,12000.0,...,195000.0,150000.0,179000.0,180000.0,197000.0,204000.0,248000.0,252000.0,61000.0,252000.0
Argentina,ARG,42000.0,45000.0,49000.0,52000.0,53000.0,207000.0,239000.0,203000.0,222000.0,239000.0,...,5737000.0,5062000.0,5779000.0,7286000.0,6648000.0,6698000.0,5689000.0,7912000.0,42000.0,7912000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Venezuela, Bolivarian Republic",VEN,6000.0,5000.0,4000.0,8000.0,8000.0,8000.0,7000.0,7000.0,7000.0,9000.0,...,111000.0,101000.0,78000.0,67000.0,67000.0,62000.0,107000.0,95000.0,6000.0,95000.0
Viet Nam,VNM,327000.0,346000.0,369000.0,394000.0,414000.0,363000.0,320000.0,257000.0,238000.0,279000.0,...,2229000.0,1781000.0,1411000.0,2213000.0,1739000.0,1491000.0,1819000.0,2519000.0,327000.0,2519000.0
Yemen,YEM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10000.0,3000.0,5000.0,2000.0,1000.0,4000.0,8000.0,5000.0,0.0,5000.0


### Other

In [8]:
fao_utl = fao[fao['DisplayNotes'] == 'Other']
dd = fao_utl[fao_utl['IFs Group'] != 'Crops']

dt = df[df['Element Code'].isin([5154,5171])]
dth = dfh[dfh['Element Code'].isin([5154,5171])]

dt = dt[dt['Item Code'].isin(dd['Item Code'])]
dth = dth[dth['Item Code'].isin(dd['Item Code'])]



for j in dd.index:
    v = dd.loc[j,'Variable']
    # m = dd.loc[j,'Element Code']
    i = dd.loc[j,'Item Code']
    u = dd.loc[j,'Units']
    if dd.isnull().loc[j,'Formula']:
        f = dd.loc[j,'Formula']
    else:
        f = "val" + str(dd.loc[j,'Formula'])
    if dd.isnull().loc[j,'Decimal Places']:
        d = dd.loc[j,'Decimal Places']
    else:
        d = int(dd.loc[j,'Decimal Places'])
    print(v,u,f,d)
    dt1 = dt[dt['Item Code'] == i]
    dt1 = dt1[['Area','Y2010', 'Y2011', 'Y2012',
        'Y2013', 'Y2014', 'Y2015', 'Y2016', 'Y2017', 'Y2018', 'Y2019', 'Y2020',
        'Y2021']
        ]
    dt1.columns = ['Area', 2010, 2011, 2012,
    2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
    2021]
    dt1 = dt1.melt(
        id_vars='Area',
        var_name='year',
        value_name = 'val')
    
    if dd.isnull().loc[j,'Formula']:
        dt1 = dt1
    else:
        dt1.val = dt1.eval(f)
    if dd.isnull().loc[j,'Decimal Places']:
        dt1 = dt1
    else:
        dt1.val = round(dt1.val, d)
    # print(dt1.shape)
    dt1 = country.merge(dt1,
                        on = 'Area',
                        how = 'left')
    dt1.drop(columns='Area',
            inplace=True)
    dth1 = dth[dth['Item Code'] == i]
    dth1 = dth1[['Area','Y1961', 'Y1962', 'Y1963',
    'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971',
    'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979',
    'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
    'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995',
    'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003',
    'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009']
        ]
    dth1.columns = ['Area', 1961, 1962, 1963,
    1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
    1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
    1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987,
    1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
    1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
    2004, 2005, 2006, 2007, 2008, 2009]
    dth1 = dth1.melt(
        id_vars='Area',
        var_name='year',
        value_name = 'val')
    if dd.isnull().loc[j,'Formula']:
        dth1 = dth1
    else:
        dth1.val = dth1.eval(f)
    if dd.isnull().loc[j,'Decimal Places']:
        dth1 = dth1
    else:
        dth1.val = round(dth1.val, d)
    dth1 = countryH.merge(dth1,
                        on = 'Area',
                        how = 'left')
    dth1.drop(columns='Area',
            inplace=True)
    # print(dth1.shape)
    
    final = pd.concat([dt1,dth1])
    final = final.groupby(['Country', 'FIPS_CODE','year']).sum(min_count=1).reset_index()
    first_last = final.sort_values(['Country','year']).groupby(final['Country'])['val'] \
        .agg(['first', 'last']).reset_index()
    first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
    final = final.pivot_table(index = ['Country', 'FIPS_CODE'],
                    columns='year',
                    values = 'val').reset_index()
    final = final.merge(first_last,
                        on = 'Country',
                        how = 'left'
                        )
    final = country.merge(final,
                        on = ['Country','FIPS_CODE'],
                        how = 'left')
    final.drop(columns='Area', inplace = True)
    final.dropna(axis = 0, how = 'all', inplace=True)
    final = final.drop_duplicates(ignore_index=True)
    print(v)
    final = final.sort_values(by='Country')
    final = final.set_index(['Country','FIPS_CODE'])
    try:
        final = final.apply(lambda x: pd.Series.round(x, d))
    except:
        final = final
    # final.to_csv(f'IFs Import/{v}.csv',
    #             #  index=False
    #              )
final

AgFishtoOtherUtilFreshwaterFAO Tonnes val*1000 4
AgFishtoOtherUtilFreshwaterFAO
AgFishtoOtherUtilDemersalFAO Tonnes val*1000 4
AgFishtoOtherUtilDemersalFAO
AgFishtoOtherUtilPelagicFAO Tonnes val*1000 4
AgFishtoOtherUtilPelagicFAO
AgFishtoOtherUtilMarineFAO Tonnes val*1000 4
AgFishtoOtherUtilMarineFAO
AgFishtoOtherUtilCrustaceansFAO Tonnes val*1000 4
AgFishtoOtherUtilCrustaceansFAO
AgFishtoOtherUtilCephalopodsFAO Tonnes val*1000 4
AgFishtoOtherUtilCephalopodsFAO
AgFishtoOtherUtilMolluscsFAO Tonnes val*1000 4
AgFishtoOtherUtilMolluscsFAO
AgFishtoOtherUtilAqAnimalsFAO Tonnes val*1000 4
AgFishtoOtherUtilAqAnimalsFAO
AgFishtoOtherUtilAqPlantsFAO Tonnes val*1000 4
AgFishtoOtherUtilAqPlantsFAO
AgFishtoOtherUtilBodyOilFAO Tonnes val*1000 4
AgFishtoOtherUtilBodyOilFAO
AgFishtoOtherUtilLiverOilFAO Tonnes val*1000 4
AgFishtoOtherUtilLiverOilFAO
AGMeattoOtherUtilFAO Tonnes val*1000 nan
AGMeattoOtherUtilFAO


,,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2014,2015,2016,2017,2018,2019,2020,2021,Earliest,MostRecent
Country,FIPS_CODE,,,,,,,,,,,,,,,,,,,,,
Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,DZA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Argentina,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Venezuela, Bolivarian Republic",VEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Viet Nam,VNM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## FAO_FBS

In [9]:
fao_fbs = fao[fao['DisplayNotes'].isnull()
              ]
fao_fbs.shape

(147, 8)

### Crops & FruVeg

In [10]:
fao_cf = fao_fbs[fao_fbs['IFs Group'].isin(['Crops','FruVeg'])]
print(fao_cf.shape)
cf_agg = pd.read_excel('FAO FBS aggregation.xlsx')
cf_agg = cf_agg[cf_agg['IFs Group'].isin(['Crops','FruVeg'])]
# print(dt['Item Code'].unique())
# cf_agg
for group in ['Crops','FruVeg']:
    dd = fao_cf[fao_cf['IFs Group'] == group]
    agg = cf_agg[cf_agg['IFs Group'] == group]
    # dd
    dt = df[df['Item Code'].isin(agg['Item Code'])]
    dth = dfh[dfh['Item Code'].isin(agg['Item Code'])]
    # print(dt['Item Code'].unique())
    # print(dt['Element Code'].unique())
    # print(dth['Item Code'].unique())
    # print(dth['Element Code'].unique())
    for j in dd.index:
        v = dd.loc[j,'Variable']
        m = dd.loc[j,'Element Code']
        u = dd.loc[j,'Units']
        if dd.isnull().loc[j,'Formula']:
            f = dd.loc[j,'Formula']
        else:
            f = "val" + str(dd.loc[j,'Formula'])
        if dd.isnull().loc[j,'Decimal Places']:
            d = dd.loc[j,'Decimal Places']
        else:
            d = int(dd.loc[j,'Decimal Places'])
        # print(v,m,u,f,d)

        dt1 = dt[dt['Element Code'] == m]
        dt1 = dt1[['Area','Y2010', 'Y2011', 'Y2012',
            'Y2013', 'Y2014', 'Y2015', 'Y2016', 'Y2017', 'Y2018', 'Y2019', 'Y2020',
            'Y2021']
            ]
        dt1.columns = ['Area', 2010, 2011, 2012,
            2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
            2021]

        dt1 = dt1.melt(
            id_vars='Area',
            var_name='year',
            value_name = 'val')
        
        if dd.isnull().loc[j,'Formula']:
            dt1 = dt1
        else:
            dt1.val = dt1.eval(f)

        if dd.isnull().loc[j,'Decimal Places']:
            dt1 = dt1
        else:
            dt1.val = round(dt1.val, d)
        # print(dt1.shape)
        dt1 = country.merge(dt1,
                            on = 'Area',
                            how = 'left')
        dt1.drop(columns='Area',
                inplace=True)

        dth1 = dth[dth['Element Code'] == m]
        dth1 = dth1[['Area','Y1961', 'Y1962', 'Y1963',
        'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971',
        'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979',
        'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
        'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995',
        'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003',
        'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009']
            ]
        dth1.columns = ['Area', 1961, 1962, 1963,
        1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
        1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
        1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987,
        1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
        1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
        2004, 2005, 2006, 2007, 2008, 2009]

        dth1 = dth1.melt(
            id_vars='Area',
            var_name='year',
            value_name = 'val')
        
        if dd.isnull().loc[j,'Formula']:
            dth1 = dth1
        else:
            dth1.val = dth1.eval(f)

        if dd.isnull().loc[j,'Decimal Places']:
            dth1 = dth1
        else:
            dth1.val = round(dth1.val, d)
        dth1 = countryH.merge(dth1,
                            on = 'Area',
                            how = 'left')
        dth1.drop(columns='Area',
                inplace=True)
        # print(dth1.shape)
        
        final = pd.concat([dt1,dth1])
        final = final.groupby(['Country', 'FIPS_CODE','year']).sum(min_count=1).reset_index()
        first_last = final.sort_values(['Country','year']).groupby(final['Country'])['val'] \
            .agg(['first', 'last']).reset_index()
        first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
        final = final.pivot_table(index = ['Country', 'FIPS_CODE'],
                        columns='year',
                        values = 'val').reset_index()
        final = final.merge(first_last,
                            on = 'Country',
                            how = 'left'
                            )
        final = country.merge(final,
                            on = ['Country','FIPS_CODE'],
                            how = 'left')
        final.drop(columns='Area', inplace = True)
        final.dropna(axis = 0, how = 'all', inplace=True)
        final = final.drop_duplicates(ignore_index=True)
        print(v)
        final = final.sort_values(by='Country')
        final = final.set_index(['Country','FIPS_CODE'])
        try:
            final = final.apply(lambda x: pd.Series.round(x, d))
        except:
            final = final
        # final.to_csv(f'IFs Import/{v}.csv',
        #             #  index=False
        #              )
    
final

(14, 8)
AGCropFoodSupplyPerCapPerDayFAO
AGCropCalPerCapPerDayFAO
AGCropProteinPerCapPerDayFAO
AGCroptoWasteFAO
AGCroptoFoodManuFAO
AGCroptoFoodFAO
AGCropDomesticSupplyFAO
AGCropProductionFAO
AGCroptoFeedFAO
AGCroptoSeedFAO
AGCropImportsFAO
AGCropExportsFAO
AgFruVegIm
AgFruVegEx


,,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2014,2015,2016,2017,2018,2019,2020,2021,Earliest,MostRecent
Country,FIPS_CODE,,,,,,,,,,,,,,,,,,,,,
Afghanistan,AFG,0.091,0.110,0.102,0.084,0.119,0.193,0.211,0.191,0.192,0.208,...,0.321,0.320,0.449,0.512,1.183,0.902,0.755,1.325,0.091,1.325
Albania,ALB,0.008,0.011,0.007,0.008,0.012,0.017,0.019,0.023,0.028,0.036,...,0.082,0.135,0.141,0.127,0.140,0.130,0.146,0.153,0.008,0.153
Algeria,DZA,0.352,0.423,0.422,0.313,0.301,0.278,0.229,0.281,0.298,0.251,...,0.046,0.060,0.066,0.066,0.063,0.115,0.136,0.088,0.352,0.088
Angola,AGO,0.001,0.003,0.003,0.004,0.005,0.009,0.009,0.018,0.023,0.041,...,0.000,0.000,0.013,0.010,0.005,0.011,0.019,0.022,0.001,0.022
Argentina,ARG,0.211,0.265,0.346,0.245,0.344,0.269,0.349,0.383,0.344,0.302,...,2.053,2.303,2.456,2.090,2.485,2.522,2.714,2.261,0.211,2.261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Venezuela, Bolivarian Republic",VEN,0.004,0.006,0.011,0.013,0.017,0.025,0.024,0.021,0.023,0.036,...,0.008,0.008,0.008,0.008,0.009,0.002,0.002,0.001,0.004,0.001
Viet Nam,VNM,0.006,0.007,0.008,0.010,0.010,0.005,0.003,0.001,0.000,0.003,...,0.708,1.387,1.691,2.046,1.897,1.549,3.134,2.039,0.006,2.039
Yemen,YEM,0.004,0.004,0.005,0.005,0.005,0.005,0.004,0.003,0.002,0.002,...,0.170,0.212,0.132,0.329,0.217,0.165,0.241,0.335,0.004,0.335


### Other

In [11]:
fao_other = fao_fbs[~fao_fbs['IFs Group'].isin(['Crops','FruVeg'])]
print(fao_other.shape)
# print(len(fao_other['Item Code'].values))
# print(len(fao_other['Element Code'].values))
dt = df
dth = dfh

(133, 8)


In [12]:
dd = fao_other
# dd = fao_other[fao_other['Variable'] == 'AgProdCereals']
for j in dd.index:
    v = dd.loc[j,'Variable']
    m = dd.loc[j,'Element Code']
    u = dd.loc[j,'Units']
    if dd.isnull().loc[j,'Formula']:
        f = dd.loc[j,'Formula']
    else:
        f = "val" + str(dd.loc[j,'Formula'])
    if dd.isnull().loc[j,'Decimal Places']:
        d = dd.loc[j,'Decimal Places']
    else:
        d = int(dd.loc[j,'Decimal Places'])
    # print(v,m,u,f,d)
    item = dd.loc[j,'Item Code']
    # print(item)
    dt1 = dt[dt['Element Code'] == m]
    dt1 = dt1[dt1['Item Code'] == item]
    dt1 = dt1[['Area','Y2010', 'Y2011', 'Y2012',
        'Y2013', 'Y2014', 'Y2015', 'Y2016', 'Y2017', 'Y2018', 'Y2019', 'Y2020',
        'Y2021']
        ]
    dt1.columns = ['Area', 2010, 2011, 2012,
        2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
        2021]
    dt1 = dt1.melt(
        id_vars='Area',
        var_name='year',
        value_name = 'val')
    
    if dd.isnull().loc[j,'Formula']:
        dt1 = dt1
    else:
        dt1.val = dt1.eval(f)
    if dd.isnull().loc[j,'Decimal Places']:
        dt1 = dt1
    else:
        dt1.val = round(dt1.val, d)
    # print(dt1.shape)
    dt1 = country.merge(dt1,
                        on = 'Area',
                        how = 'left')
    dt1.drop(columns='Area',
            inplace=True)
    dth1 = dth[dth['Element Code'] == m]
    dth1 = dth1[dth1['Item Code'] == item]
    dth1 = dth1[['Area','Y1961', 'Y1962', 'Y1963',
    'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971',
    'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979',
    'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
    'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995',
    'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003',
    'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009']
        ]
    dth1.columns = ['Area', 1961, 1962, 1963,
    1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
    1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
    1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987,
    1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
    1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
    2004, 2005, 2006, 2007, 2008, 2009]
    dth1 = dth1.melt(
        id_vars='Area',
        var_name='year',
        value_name = 'val')
    
    if dd.isnull().loc[j,'Formula']:
        dth1 = dth1
    else:
        dth1.val = dth1.eval(f)
    if dd.isnull().loc[j,'Decimal Places']:
        dth1 = dth1
    else:
        dth1.val = round(dth1.val, d)
    dth1 = countryH.merge(dth1,
                        on = 'Area',
                        how = 'left')
    dth1.drop(columns='Area',
            inplace=True)
    # print(dth1.shape)
    
    final = pd.concat([dt1,dth1])
    final = final.groupby(['Country', 'FIPS_CODE','year']).sum(min_count=1).reset_index()
    first_last = final.sort_values(['Country','year']).groupby(final['Country'])['val'] \
        .agg(['first', 'last']).reset_index()
    first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
    final = final.pivot_table(index = ['Country', 'FIPS_CODE'],
                    columns='year',
                    values = 'val').reset_index()
    final = final.merge(first_last,
                        on = 'Country',
                        how = 'left'
                        )
    final = country.merge(final,
                        on = ['Country','FIPS_CODE'],
                        how = 'left')
    final.drop(columns='Area', inplace = True)
    final.dropna(axis = 0, how = 'all', inplace=True)
    final = final.drop_duplicates(ignore_index=True)
    print(v, dt1.shape, dth1.shape)
    final = final.sort_values(by='Country')
    final = final.set_index(['Country','FIPS_CODE'])
    try:
        final = final.apply(lambda x: pd.Series.round(x, d))
    except:
        final = final
    # final.to_csv(f'IFs Import/{v}.csv',
    #             #  index=False
    #              )
final

AgFishFoodSupplyPerCapPerDayFreshwaterFAO (2158, 4) (8595, 4)
AgFishFoodSupplyPerCapPerDayDemersalFAO (2158, 4) (8547, 4)
AgFishFoodSupplyPerCapPerDayPelagicFAO (2158, 4) (8547, 4)
AgFishFoodSupplyPerCapPerDayMarineFAO (2147, 4) (8499, 4)
AgFishFoodSupplyPerCapPerDayCrustaceansFAO (2147, 4) (8499, 4)
AgFishFoodSupplyPerCapPerDayCephalopodsFAO (2147, 4) (8355, 4)
AgFishFoodSupplyPerCapPerDayMolluscsFAO (2147, 4) (8451, 4)
AgFishFoodSupplyPerCapPerDayAqAnimalsFAO (2103, 4) (6675, 4)
AgFishFoodSupplyPerCapPerDayAqPlantsFAO (233, 4) (7875, 4)
AgFishFoodSupplyPerCapPerDayBodyOilFAO (310, 4) (7827, 4)
AgFishFoodSupplyPerCapPerDayLiverOilFAO (739, 4) (7203, 4)
AGMeatFoodSupplyPerCapPerDayFAO (2158, 4) (8595, 4)
AgFishCalPerCapPerDayFreshwaterFAO (2158, 4) (8595, 4)
AgFishCalPerCapPerDayDemersalFAO (2158, 4) (8547, 4)
AgFishCalPerCapPerDayPelagicFAO (2158, 4) (8547, 4)
AgFishCalPerCapPerDayMarineFAO (2147, 4) (8499, 4)
AgFishCalPerCapPerDayCrustaceansFAO (2147, 4) (8499, 4)
AgFishCalPerCapPerD

,,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2014,2015,2016,2017,2018,2019,2020,2021,Earliest,MostRecent
Country,FIPS_CODE,,,,,,,,,,,,,,,,,,,,,
Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Albania,ALB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,1000.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0
Algeria,DZA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,2000.0,1000.0,0.0,0.0,0.0
Angola,AGO,3000.0,2000.0,2000.0,2000.0,2000.0,3000.0,3000.0,3000.0,2000.0,2000.0,...,NaN,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,3000.0,0.0
Argentina,ARG,524000.0,648000.0,878000.0,688000.0,582000.0,730000.0,793000.0,667000.0,874000.0,829000.0,...,567000.0,450000.0,441000.0,539000.0,688000.0,983000.0,997000.0,880000.0,524000.0,880000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Venezuela, Bolivarian Republic",VEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,1000.0
Viet Nam,VNM,0.0,0.0,0.0,1000.0,2000.0,0.0,0.0,0.0,0.0,0.0,...,17000.0,25000.0,43000.0,30000.0,47000.0,53000.0,38000.0,54000.0,0.0,54000.0
Yemen,YEM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0


## FAO_Land

In [13]:
# LandPotentialArable
dt = pd.read_csv('Potential_Arable_Land.csv')
countryL = pd.read_excel('CountryL.xlsx')
dt = countryL.merge(dt,
                    on = 'Area',
                    how = 'left')
dt.drop(columns='Area', inplace=True)
dt.Year = dt.Year.astype("Int64")

dt = dt[['Country','FIPS_CODE','Year','Value']]
dt = dt.groupby(['Country','FIPS_CODE','Year']).sum(min_count=1).reset_index()
dt.Value = round(dt.Value, 0)
first_last = dt.sort_values(['Country','Year']).groupby(dt['Country'])['Value'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
# pd.DataFrame({'Area':list(dt.Area.unique())}).to_excel('CountryL.xlsx', index = False)
dt = dt.pivot_table(index = ['Country','FIPS_CODE'],
                    columns='Year',
                    values = 'Value').reset_index()
dt = dt.merge(first_last,
                on = 'Country',
                how = 'left')
dt = countryL.merge(dt,
                    on = ['Country','FIPS_CODE'],
                    how = 'left')
dt.drop(columns='Area', inplace=True)
dt.dropna(axis = 0, how = 'all', inplace=True)
dt = dt.drop_duplicates(ignore_index=True)
dt = dt.set_index(['Country','FIPS_CODE'])
dt = dt.apply(lambda x: pd.Series.round(x, 0))
# dt.to_csv('IFs Import/LandPotentialArable.csv', 
#         #   index = False
#           )
dt

,,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2014,2015,2016,2017,2018,2019,2020,2021,Earliest,MostRecent
Country,FIPS_CODE,,,,,,,,,,,,,,,,,,,,,
Afghanistan,AFG,7650.0,7700.0,7750.0,7800.0,7800.0,7835.0,7844.0,7844.0,7860.0,7870.0,...,7771.0,7765.0,7729.0,7699.0,7794.0,7788.0,7829.0,7829.0,7650.0,7829.0
Albania,ALB,432.0,436.0,440.0,443.0,460.0,475.0,488.0,500.0,510.0,521.0,...,616.0,615.0,620.0,612.0,611.0,610.0,600.0,600.0,432.0,600.0
Algeria,DZA,6472.0,6300.0,6200.0,6132.0,6203.0,6221.0,6301.0,6243.0,6233.0,6248.0,...,7469.0,7462.0,7404.0,7471.0,7501.0,7531.0,7531.0,7531.0,6472.0,7531.0
Angola,AGO,2670.0,2700.0,2720.0,2750.0,2770.0,2800.0,2830.0,2860.0,2900.0,2900.0,...,5010.0,5125.0,5347.0,5352.0,5357.0,5363.0,5368.0,5373.0,2670.0,5373.0
Argentina,ARG,18597.0,19180.0,19600.0,20000.0,20461.0,21608.0,22341.0,23851.0,23977.0,24103.0,...,39216.0,40339.0,40162.0,40386.0,40209.0,40809.0,42009.0,42209.0,18597.0,42209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Venezuela, Bolivarian Republic",VEN,2900.0,2900.0,2903.0,2907.0,2911.0,2913.0,2922.0,2919.0,2924.0,2931.0,...,2600.0,2600.0,2600.0,2600.0,2600.0,2600.0,2600.0,2600.0,2900.0,2600.0
Viet Nam,VNM,5550.0,5550.0,5550.0,5550.0,5550.0,5550.0,5570.0,5590.0,5600.0,5630.0,...,6410.0,7003.0,6998.0,6988.0,6952.0,6784.0,6787.0,6787.0,5550.0,6787.0
Yemen,YEM,1262.0,1273.0,1285.0,1295.0,1305.0,1310.0,1317.0,1322.0,1327.0,1332.0,...,1154.0,1138.0,1096.0,1163.0,1167.0,1158.0,1158.0,1158.0,1262.0,1158.0
